In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
def shuffle_df(df):
    return df.sample(frac=1).reset_index(drop=True)


def get_split(df, train_size=0.5, test_size=0.4):
    train, rest = train_test_split(df, train_size=train_size)
    test, val = train_test_split(rest, train_size=test_size/train_size)
    return train, test, val


def train_test_val(df, label_name, labels, train_size=0.5, test_size=0.4):
    df_train, df_test, df_val = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for lab in labels:
        df_lab = df[df[label_name]==lab]
        tr, te, val = get_split(df_lab, train_size, test_size)
        df_train = df_train.append(tr)
        df_test = df_test.append(te)
        df_val = df_val.append(val)
        
    df_train['split'] = 'train'
    df_test['split'] = 'test'
    df_val['split'] = 'val'
    df = pd.DataFrame()
    df = df.append(shuffle_df(df_train))
    df = df.append(shuffle_df(df_test))
    df = df.append(shuffle_df(df_val))
    print(df)
    return df

In [ ]:
cwd = os.getcwd()
root = os.path.join(cwd, '..')
imdb_file_path = os.path.join(root, 'datasets/imdb/IMDB.csv')

df = pd.read_csv(imdb_file_path)
df = train_test_val(df, 'sentiment', ['positive', 'negative'])

In [ ]:
from preprocessing import imdb_preprocess

df['review'] = df['review'].apply(lambda x: imdb_preprocess(x))
save_path = os.path.join(root, 'datasets/imdb/imdb_clean_split.csv')
df.to_csv(save_path)